<a href="https://www.kaggle.com/code/dhruv836/slide-revamp?scriptVersionId=287862086" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install python-pptx lxml


In [ ]:
!pip install torch torchvision==0.15.2 torchaudio --index-url https://download.pytorch.org/whl/cu118


In [ ]:
#!pip install torchvision==0.15.2 --force-reinstall --no-cache-dir


run the above cell first because,it will install a version of numpy which is not compatible with our models so the next will change to appropraite version

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install numpy==1.26.4 scipy==1.11.4
!pip install matplotlib==3.8.2
!pip install filterpy facexlib basicsr


In [ ]:

import os
os.listdir("/kaggle/working/")




In [ ]:
# for taking input
PPTX_PATH = "/kaggle/input/sample-4/sample4.pptx"
USER_PROMPT = "colorful"



In [ ]:

assert os.path.exists(PPTX_PATH)
print("PPTX found ")


In [ ]:
!git clone https://github.com/dhruv8361343/SlideRevamp.git

In [ ]:
import sys
sys.path.append("/kaggle/working/SlideRevamp")


In [ ]:
from pptx_extractor import extract
from content_refinement import refine_content

print("Imports successful ")


In [ ]:
from pptx_extractor import extract
from pathlib import Path

OUTPUT_DIR = Path("/kaggle/working/outputs")

extract(
    pptx_path=PPTX_PATH,
    out_dir=OUTPUT_DIR
)
print("don")


In [ ]:
STYLE_TOKEN = refine_content(
    ingestion_output_dir="/kaggle/working/outputs",
    user_vibe=USER_PROMPT
)


In [ ]:
# for saving our style token that we call in previous code block
import json
from pathlib import Path

STYLE_PATH = Path("/kaggle/working/outputs/style_token.json")
STYLE_PATH.write_text(
    json.dumps(STYLE_TOKEN, indent=2),
    encoding="utf-8"
)

print("Style token ")


now running the Real-ESRGAN model for image scaling can be used for upscaling the bbackground generated

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git


In [ ]:
!pip install -r Real-ESRGAN/requirements.txt --no-deps


In [ ]:
!pip install -e Real-ESRGAN


for saving weights from input to output folder

In [ ]:
!mkdir -p /kaggle/working/Real-ESRGAN/weights
!cp /kaggle/input/weights/RealESRGAN_x4plus.pth /kaggle/working/Real-ESRGAN/weights/


In [ ]:
!python Real-ESRGAN/inference_realesrgan.py -h


In [ ]:
from image_processing.upscale import upscale_image

test_img = "/kaggle/working/outputs/images/slide02_shape3_img.png"

upscaled = upscale_image(
    image_path=test_img,
    output_dir="/kaggle/working/outputs/images_upscaled"
)

print("Upscaled image:", upscaled)


now the u-2-net model for image cropping(it provides mask for the most imp part of image and we will crop it using python script)

In [ ]:
!git clone https://github.com/xuebinqin/U-2-Net.git

transfering weights form input to output directory


In [ ]:
!mkdir -p /kaggle/working/U-2-Net/saved_models/u2net
!cp /kaggle/input/weights/u2net.pth /kaggle/working/U-2-Net/saved_models/u2net


masking using u-2-net

In [ ]:
from image_processing.mask import generate_mask

mask = generate_mask(
    image_path=upscaled,
    output_dir="/kaggle/working/outputs/masks"
)

print("Mask generated:", mask)


cropping 

In [ ]:
from image_processing.smart_crop import smart_crop
import cv2

final = smart_crop(
    image_path=str(upscaled),
    mask_path=str(mask)
)

cv2.imwrite(
    "/kaggle/working/outputs/images_final/final.png",
    final
)


for running on all slides

In [ ]:
from pathlib import Path
from image_processing.upscale import upscale_image
from image_processing.mask import generate_mask
from image_processing.smart_crop import smart_crop
import cv2

# Directories
INPUT_IMAGES_DIR = Path("/kaggle/working/outputs/images")
UPSCALED_DIR = Path("/kaggle/working/outputs/images_upscaled")
MASKS_DIR = Path("/kaggle/working/outputs/masks")
FINAL_DIR = Path("/kaggle/working/outputs/images_final")

# Ensure output dirs exist
UPSCALED_DIR.mkdir(parents=True, exist_ok=True)
MASKS_DIR.mkdir(parents=True, exist_ok=True)
FINAL_DIR.mkdir(parents=True, exist_ok=True)

# Process every extracted image
for img_path in INPUT_IMAGES_DIR.glob("*"):
    print(f"\nProcessing: {img_path.name}")

    # Upscale (Real-ESRGAN)
    upscaled_path = upscale_image(
        image_path=img_path,
        output_dir=UPSCALED_DIR
    )

    #  Generate U-2-Net mask
    mask_path = generate_mask(
        image_path=upscaled_path,
        output_dir=MASKS_DIR
    )

    #  Smart crop
    final_img = smart_crop(
        image_path=str(upscaled_path),
        mask_path=str(mask_path),
        target_ratio=16/9
    )

    #  Save final image
    final_out = FINAL_DIR / img_path.name
    cv2.imwrite(str(final_out), final_img)

    print(f" Final image saved: {final_out}")


now using the stable diffusion v1.5 (fine tuned using LoRA using kohya_ss notebook)for background generation using the prompt from LLM

In [ ]:
# 1. Upgrading the conflicting library
!pip install peft==0.10.0 --upgrade

# 2. Install your required libraries
!pip install \
  diffusers==0.25.1 \
  transformers==4.36.2 \
  accelerate==0.25.0 \
  huggingface_hub==0.20.3 \
  safetensors==0.4.2


In [ ]:
import torch
import accelerate.utils.memory as mem
from diffusers import StableDiffusionPipeline

# --- 1. Apply the Cache Patch (Keep this if your environment needs it) ---
if not hasattr(mem, "clear_device_cache"):
    def clear_device_cache():
        torch.cuda.empty_cache() # Explicitly empty torch cache
    mem.clear_device_cache = clear_device_cache
print("Patched clear_device_cache ✔")

# --- 2. Load the Pipeline ---
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None, # Warning: Be careful with generated content
    use_safetensors=True # Recommended for speed and safety
).to("cuda")



print("Pipeline loaded successfully ✔")

loading our weights which we had get after finetuning

In [ ]:
pipe.load_lora_weights(
    "/kaggle/input/weight/last.safetensors"
)


In [ ]:
import json

with open("/kaggle/working/outputs/style_token.json", "r") as f:
    style_token = json.load(f)

bg_prompt = style_token["background_prompt"]
print(bg_prompt)


In [ ]:
image = pipe(
    prompt=bg_prompt,
    negative_prompt=(
        "text, letters, words, logo, watermark, "
        "people, faces, objects, icons"
    ),
    num_inference_steps=25,
    guidance_scale=7.5,
    height=544,
    width=544
).images[0]


In [ ]:
from pathlib import Path

bg_dir = Path("/kaggle/working/outputs/backgrounds")
bg_dir.mkdir(parents=True, exist_ok=True)

bg_path = bg_dir / "slide_background.png"
image.save(bg_path)

print("Background saved:", bg_path)


In [ ]:
from image_processing.upscale import upscale_image

back_img = "/kaggle/working/outputs/backgrounds/slide_background.png"

upscaled = upscale_image(
    image_path=back_img,
    output_dir="/kaggle/working/outputs/backgrounds"
)

print("Upscaled image:", upscaled)